In [ ]:
# Primeiro, realizamos a instalação do mlflow e autoML do sklearn

#!pip install auto-sklearn

!pip install --upgrade pip

!pip install mlflow

!pip install tqdm 

!pip install statsmodels

In [ ]:
import mlflow
import requests

url = "http://checkip.amazonaws.com"
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    IP = response.text.replace('\n', '')
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

uri_mlflow = "http://"+IP+":8089"
print("Configurando servidor do MLFlow: ")
print(uri_mlflow)

mlflow.set_tracking_uri(uri_mlflow)

In [ ]:
# MODELO DE CLASSIFICAÇÃO COM ALGUMAS COLUNAS

import pandas as pd
from sklearn.model_selection import train_test_split

# LE O DATASET COMO UM PANDAS DATAFRAME.
df_data = pd.read_csv('curso_ml.csv')

# Definição das colunas que serão features
features = [
    #"MATRICULA", NOME
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    #"INGLES_DESC", # usando algumas colunas
    #"CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data[features]
y = df_data[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

print("Colunas dos Dados de treinamento : (70%)")
print("")
print(X_train.info())
print("")
print(y_train.info())
print("")


In [ ]:
# TREINAMENTO REALIZADO ANTERIORMENTE:

# Método para criação de modelos baseados em árvores de decisão
from sklearn.tree import DecisionTreeClassifier

# Criação de uma árvore de decisão com a biblioteca ``scikit-learn``:
decision_tree = DecisionTreeClassifier()

# Treino do modelo (é chamado o método *fit()* com os conjuntos de treino)
decision_tree.fit(
    X_train,
    y_train
)

# Realização de teste cego no modelo criado
y_pred = decision_tree.predict(X_test)

from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

In [ ]:
# Definindo a função para subir as informações do modelo no MLFLOW:

import mlflow
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score
import warnings
import logging

# Reduz o nível de log do MLflow para ERROR (ignora WARNING)
logging.getLogger("mlflow").setLevel(logging.ERROR)

def log_model_to_mlflow(model, X_train, y_test, y_pred, experiment_name, model_name=None, tag=None):
    """
    Loga um modelo no MLflow com métricas, hiperparâmetros e assinatura.
    
    Args:
        model: Modelo treinado.
        X_train: Dados de entrada de treino.
        y_test: Valores verdadeiros do conjunto de teste.
        y_pred: Predições do modelo no conjunto de teste.
        experiment_name: Nome do experimento no MLflow.
        tag: Tag descritiva para o run (opcional).
    """
    accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.set_experiment(experiment_name)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suprime todos os warnings dentro deste bloco

        with mlflow.start_run():
            # Extrai e loga os hiperparâmetros automaticamente
            params = model.get_params()
            mlflow.log_params(params)

            # Loga a métrica
            mlflow.log_metric("accuracy", accuracy)

            # Adiciona tag, se houver
            if tag:
                mlflow.set_tag("Training Info", tag)

            # Inferência de assinatura
            y_train.name = "PERFIL"
            signature = infer_signature(X_train, y_train)
            #signature = infer_signature(X_train, model.predict(X_train))

            # Usa nome do modelo com base no algoritmo
            # Detecta o nome do melhor modelo AutoML se aplicável
            #if model.__class__.__name__ == "AutoSklearnClassifier":
            try:
                print("Verificando o nome do modelo: ")
                leaderboard = model.leaderboard()
                best_model_name = leaderboard.iloc[0]["type"]
                print("Nome do modelo = "+best_model_name)
                model_name = f"{best_model_name}_automl_model"
            except Exception:
                model_name = f"{model.__class__.__name__}_automl_model"
            #else:
            #    model_name = f"{model.__class__.__name__}_automl_model"
                    
            # Define nome do RUN
            mlflow.set_tag("mlflow.runName", model_name)

            # Log do modelo
            mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path=model_name,
                signature=signature,
                input_example=X_train,
                registered_model_name=model_name,
            )
        
print("Definida a função para subir as informações do modelo no MLFLOW")

In [ ]:
# Subindo Modelo Versão 1 apra o MLFLOW

log_model_to_mlflow(
    model=decision_tree,
    X_train=X_train,
    y_test=y_test,
    y_pred=y_pred,
    experiment_name="Experimento MODELO CURSO",
    tag="Nome do Modelo e Hiperparâmetros extraídos automaticamente"
)

print("Criado Modelo no MLFLOW com Acurácia de: {:.4f}%".format(100 * accuracy_score(y_test, y_pred)))

In [ ]:
import autosklearn.classification
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import threading
import time


# Parâmetros do AutoML
total_time = 300  # Tempo total (em segundos)
per_model_time = 30  # Tempo máximo por modelo

# Criando uma barra de progresso em thread separada
def show_progress_bar(duration):
    for _ in tqdm(range(duration), desc="AutoML training progress", ncols=100):
        time.sleep(1)

# Iniciando barra de progresso em segundo plano
progress_thread = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread.start()

# Treinamento com auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/autosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

automl.fit(X_train, y_train)

# Espera barra terminar se for mais rápida
progress_thread.join()

# Resultados
print("\nRanking dos modelos testados DE CLASSIFICAÇÃO COM ALGUMAS COLUNAS: \n")
print(automl.leaderboard())

y_pred = automl.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("")
print(f"\nAcurácia: {accuracy:.4f}")


In [ ]:
# Subindo Modelo Versão AUTOML para o MLFLOW

log_model_to_mlflow(
    model=automl,
    X_train=X_train,
    y_test=y_test,
    y_pred=y_pred,
    experiment_name="Experimento MODELO CURSO",
    tag="Nome do Modelo e Hiperparâmetros extraídos automaticamente"
)

print("Criado Modelo no MLFLOW com Acurácia de: {:.4f}%".format(100 * accuracy_score(y_test, y_pred)))

In [ ]:
# MODELO DE CLASSIFICAÇÃO COM NOVAS COLUNAS TRANSFORMADAS

# Definição das colunas que serão features 
features = [
    #"MATRICULA", NOME
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    #"INGLES_DESC", # usando todas as colunas
    "CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Converte tipo da coluna de Object para String # TODO
#columns_to_convert = ["INGLES_DESC", "CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC"]
#df_data[columns_to_convert] = df_data[columns_to_convert].astype('str')

                      
# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data[features]
y = df_data[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

print("Colunas dos Dados de treinamento : (70%)")
print(X_train.info())
print("")
print(y_train.info())
print("")

In [ ]:
# Iniciando barra de progresso em segundo plano
progress_thread = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread.start()

# Treinamento com auto-sklearn
automl_cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/clsautosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

automl_cls.fit(X_train, y_train)

# Espera barra terminar se for mais rápida
progress_thread.join()

# Resultados
print("\nRanking dos modelos testados DE CLASSIFICAÇÃO COM NOVAS COLUNAS TRANSFORMADAS: \n")
print(automl_cls.leaderboard())

y_pred = automl_cls.predict(X_test)
print("")
print(f"\nAcurácia: {accuracy_score(y_test, y_pred):.4f}")

In [ ]:
# Subindo Modelo Versão AUTOML_CLS NOVAS COLUNAS para o MLFLOW

log_model_to_mlflow(
    model=automl_cls,
    X_train=X_train,
    y_test=y_test,
    y_pred=y_pred,
    experiment_name="Experimento MODELO CURSO",
    tag="Nome do Modelo e Hiperparâmetros extraídos automaticamente"
)

print("Criado Modelo no MLFLOW com Acurácia de: {:.4f}%".format(100 * accuracy_score(y_test, y_pred)))

In [ ]:
## Verificar ML FLOW

print("Acessar o servidor do MLFlow e verificar as diferentes versões criadas: ")
print(uri_mlflow)